# ScanObjectNN

> A Dataset structure for ScanObjectNN dataset.

In [ ]:
#| default_exp datasets/scanobjectnn

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pclab.transforms import *
import h5py
import numpy as np
import os

In [ ]:
#| export
class ScanObjectNN(Dataset):
    "A dataset structure to load the ScanObjectNN dataset" 
    def __init__(self, path:str,                # path of the `h5_files` folder (`.../ScanObjectNN/h5_files/`)
                       subset:str,              # `train` or `test` set
                       difficulty:str='normal', # return the variant based on the difficulty (options: `normal`, `hard`)
                       background:bool=True,    # return the versions with the background or without
                       transforms=[]):          # transforms to apply to the data for augmentation
        self.path, self.subset, self.difficulty, self.background, self.transforms = path, subset, difficulty, background, transforms
        
        assert subset in ['train', 'test']
        assert difficulty in ['normal', 'hard']
        
        # creating the path based on the background choice
        self.path = os.path.join(self.path, 'main_split') if background else os.path.join(self.path, 'main_split_nobg')
        
        # creating the actual filename based on the the difficulty and the train/test subset
        filename = 'objectdataset'
        filename = 'training_'+filename if subset=='train' else 'test_'+filename
        filename = filename if difficulty=='normal' else filename+'_augmentedrot_scale75'
        filename += '.h5'
        
        # Loading the data
        h5 = h5py.File(os.path.join(self.path, filename), 'r')
        self.points = np.array(h5['data']).astype(np.float32)
        self.labels = np.array(h5['label']).astype(int)
        h5.close()
        
        
    def __len__(self):
        return self.points.shape[0]
    
    def __getitem__(self, idx):
        
        pc = self.points[idx]
        label = self.labels[idx]
        
        for t in self.transforms:
            pc = t(pc)
            
        return pc, label

In [ ]:
#| hide
path = "/home/vvr/Desktop/vlassisgiannis/new_exps/data/ScanObjectNN/h5_files"

In [ ]:
#| eval: false
dataset = ScanObjectNN(path, 'train')
print(f'Variant with {len(dataset)} samples, with shape {dataset[0][0].shape}')

Variant with 2309 samples, with shape (2048, 3)


In [ ]:
#| eval: false
transforms = [FirstKPointsKeep(1024), AnisotropicScale(), UnitSphereNormalization()]
dataset = ScanObjectNN(path, 'train', background=False, transforms=transforms)
print(f'Variant with {len(dataset)} samples, with shape {dataset[0][0].shape}')

Variant with 2309 samples, with shape (1024, 3)


In [ ]:
#| eval: false
dataset = ScanObjectNN(path, 'train', background=False, difficulty='hard')
print(f'Variant with {len(dataset)} samples, with shape {dataset[0][0].shape}')

Variant with 11416 samples, with shape (2048, 3)


In [ ]:
#| hide 
path = "/home/vvr/Desktop/vlassisgiannis/new_exps/data/ScanObjectNN/h5_files/main_split"

In [ ]:
#| export
class ScanObjectNN_normal(Dataset):
    "A dataset structure to load the normal variant of the ScanObjectNN dataset." 
    def __init__(self, path:str,         # path of the folder that contains the `.h5` files. 
                       subset:str,       # `train` or `test` set
                       transforms=[]):   # transfors to apply to the data for augmentation
        self.path, self.subset, self.transforms = path, subset, transforms
        assert subset in ['train', 'test']
        
        if self.subset == "train":  
            h5 = h5py.File(os.path.join(self.path, 'training_objectdataset.h5'), 'r')
            self.points = np.array(h5['data']).astype(np.float32)
            self.labels = np.array(h5['label']).astype(int)
            h5.close()
        elif self.subset == "test":
            h5 = h5py.File(os.path.join(self.path, 'test_objectdataset.h5'), 'r')
            self.points = np.array(h5['data']).astype(np.float32)
            self.labels = np.array(h5['label']).astype(int)
            h5.close()

    def __len__(self):
        return self.points.shape[0]

    
    def __getitem__(self, idx):
        pt_idxs = np.arange(0, self.points.shape[1]) #2048
        if self.subset == 'train':
            np.random.shuffle(pt_idxs)

        current_points = self.points[idx, pt_idxs].copy()

        for t in self.transforms:
            current_points = t(current_points)
            
        label = self.labels[idx]
        
        return current_points, np.array([label])
        

In [ ]:
#| eval:false
dataset = ScanObjectNN_normal(path, 'train')
print(f'Variant with {len(dataset)} samples, with shape {dataset[0][0].shape}')

Variant with 2309 samples, with shape (2048, 3)


In [ ]:
#| export 
class ScanObjectNN_hardest(ScanObjectNN):
    "A dataset structure to load the hardest variant of the ScanObjectNN dataset"
    
    def __init__(self, path:str, subset:str, transforms=[]):
        self.path, self.subset, self.transforms = path , subset, transforms
        assert subset in ['train', 'test']
        
        if self.subset == 'train':
            h5 = h5py.File(os.path.join(self.path, 'training_objectdataset_augmentedrot_scale75.h5'), 'r')
            self.points = np.array(h5['data']).astype(np.float32)
            self.labels = np.array(h5['label']).astype(int)
            h5.close()
        elif self.subset == 'test':
            h5 = h5py.File(os.path.join(self.path, 'test_objectdataset_augmentedrot_scale75.h5'), 'r')
            self.points = np.array(h5['data']).astype(np.float32)
            self.labels = np.array(h5['label']).astype(int)
            h5.close()

In [ ]:
#| eval:false
transforms = [FirstKPointsKeep(1024), AnisotropicScale(), UnitSphereNormalization()]
dataset = ScanObjectNN_hardest(path, 'train', transforms=transforms)
print(f'Variant with {len(dataset)} samples, with shape {dataset[0][0].shape}')

Variant with 11416 samples, with shape (1024, 3)
